In [1]:
# Testing ollama embedding using phi-3.5-mini https://ollama.com/blog/embedding-models
# compares direct query and HyDE query

# steps to install phi-3.5-mini on ollama
# 1. install ollama
# 2. run >ollama pull phi3.5

# phi-3.5-mini embedding query results are wrong from both direct query and HyDE query!!!
# the results is ['5', '0', '4', '1', '2', '3'] from direct query and ['5', '1', '4', '0', '2', '3'] from HyDE query
# mxbai-embed-large performs for embedding.

import ollama
import chromadb

# embedding_mode = "phi3.5"
embedding_mode = "mxbai-embed-large"

documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

client = chromadb.Client()
collection = client.get_or_create_collection(name="docs"+embedding_mode)

# store each document in a vector embedding database
for i, d in enumerate(documents):
  response = ollama.embeddings(model=embedding_mode, prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d]
  )

In [2]:
collection.peek(), collection.count()

({'ids': ['0', '1', '2', '3', '4', '5'],
  'embeddings': array([[ 5.16468108e-01,  1.47336042e+00,  6.79911852e-01, ...,
           5.08700848e-01,  5.70794940e-01,  5.46187699e-01],
         [ 9.86270607e-01,  1.18459761e+00,  8.43585491e-01, ...,
          -4.44508046e-01,  9.26456213e-01,  7.99377680e-01],
         [ 7.97073603e-01,  8.96159768e-01,  7.72400081e-01, ...,
          -6.88998342e-01,  9.92778391e-02,  8.37223232e-02],
         [ 8.87079477e-01,  8.14557731e-01,  3.18510115e-01, ...,
          -5.95363855e-01,  3.43608677e-01, -3.05887222e-01],
         [ 8.60674456e-02,  8.81301284e-01, -8.46746936e-02, ...,
          -2.14637652e-01,  1.51675999e-01, -4.76360321e-04],
         [ 7.36328959e-01,  9.84707236e-01,  7.13119879e-02, ...,
          -4.18436825e-01,  3.43871683e-01, -1.19044989e-01]]),
  'metadatas': [None, None, None, None, None, None],
  'documents': ["Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
 

In [3]:
collection.get(ids=["0"])

{'ids': ['0'],
 'embeddings': None,
 'metadatas': [None],
 'documents': ["Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels"],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [4]:
# an example prompt
prompt = "What animals are llamas related to?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model=embedding_mode
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=1
)
data = results['documents'][0][0]

In [5]:
results

{'ids': [['0']],
 'distances': [[79.3682632446289]],
 'metadatas': [[None]],
 'embeddings': None,
 'documents': [["Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels"]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [6]:
show_distances = collection.query(response["embedding"], include=["distances"], n_results=6)
show_distances

{'ids': [['0', '1', '4', '3', '5', '2']],
 'distances': [[79.3682632446289,
   184.64312744140625,
   198.3860626220703,
   220.77955627441406,
   221.32534790039062,
   224.84298706054688]],
 'metadatas': None,
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None,
 'included': ['distances']}

In [7]:
# the result should be the first document: Llamas are members of the camelid family meaning ...

# use HyDE to create a tempate answer from the question.
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

hyde_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user queries succinctly."),
    ("user", "Question: {question}")
])
output_parser=StrOutputParser()
llm = ChatOllama(model="phi3.5", temperature=0)
chain = hyde_prompt | llm | output_parser

converted_q = chain.invoke({'question': prompt})
converted_q

'Llamas are most closely related to camels and other members of the Camelidae family, such as alpacas, guanacos, and vicuñas. They share a common ancestor that lived around 35 million years ago in North America before migrating into South America where they evolved independently for millions more years.'

In [8]:
# query documents by the converted_q
converted_response = ollama.embeddings( prompt=converted_q, model=embedding_mode)
converted_results = collection.query(query_embeddings=[converted_response["embedding"]], n_results=1)
converted_results

{'ids': [['0']],
 'distances': [[78.7081298828125]],
 'metadatas': [[None]],
 'embeddings': None,
 'documents': [["Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels"]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [9]:
show_converted_distances = collection.query(converted_response["embedding"], include=["distances"], n_results=6)
show_converted_distances

{'ids': [['0', '1', '5', '4', '2', '3']],
 'distances': [[78.7081298828125,
   154.53427124023438,
   161.29637145996094,
   195.13973999023438,
   201.42514038085938,
   206.93064880371094]],
 'metadatas': None,
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None,
 'included': ['distances']}

In [10]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="phi3.5",
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])

Llamas are primarily related to two other species within the Camelidae family, namely vicuñas and various types of true camels (such as dromedary camels and Bactrian camels). Here's how they connect:

1. Llama vs Vicuna: Both llamas and vicunas are native South American mammals belonging to the same Camelidae family, which also includes their distant relatives like alpacas. While llama is larger with a domesticated history used by humans for transportation as well as wool production, smaller in size than camels but bigger compared to other livestthy animals such as cattle or sheep, vicuna are wild mountain goats known primarily for the extremely fine and valuable fiber they produce – their coat.
  
2. Llama vs Camel: True camels (like dromedary or Bactrian ones) share an evolutionarily closer ancestry to llamas than any other mammals since both llama species originated from South American landmass, which used to be connected with the Middle East where true camels have their roots. Alth